# Test inference on the fine-tuned, merged model.

This notebook requires that you have first run the `fine_tune.ipynb` notebook to generate the `merged-fine-tuned` model and tokenizer. This is a utility notebook to load the new model and run inference a few times.

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

In [ ]:
merged_model_dir = "merged-fine-tuned"
#merged_model_dir = "mistralai/Mistral-7B-Instruct-v0.1"

device_map = {"": 0}

device = "cuda"

In [ ]:
merged_model = AutoModelForCausalLM.from_pretrained(
    merged_model_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

tokenizer = AutoTokenizer.from_pretrained(merged_model_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
def send_to_model(msg):
    messages = [
        {"role": "user", "content": msg},
    ]
    
    encoded = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    
    generated_ids = merged_model.generate(encoded, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id, temperature=0.4,  repetition_penalty=1.20)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return decoded

def send_chat(msg):
    result = send_to_model(msg)[0]
    return result.rsplit(" [/INST] ", 1)[1]

def print_chat(msg):
    print(send_chat(msg))

In [ ]:
print_chat("How much wood would a woodchuck chuck if a woodchuck could chuck wood?")

In [ ]:
print_chat("How do I make a bulleted list in markdown?")

In [ ]:
print_chat("What is your favorite magazine?")

In [ ]:
print_chat("If you hade 3 plates and you ate off of one of them, how many plates would you have left? Explain it step by step.")

In [ ]:
document = """
Good evening to everyone,

In partnership with Wizbang ISD, ACME will be requiring that school pick-up traffic for Boba Fett Elementary and Ashoka Tano Middle School utilize the school pick up lines going forward and cease using the ACME parking lot for pick-up in any capacity. This is to ensure that District approved pick-up safety plans are followed and provides the safest avenue for student pick-up after school. The Mos Eisley County Sheriff's Office will be on-site for the next two weeks at ACME to redirect all traffic relating to school pick-ups to the appropriate school pick-up line as well as will be giving out citations for standing in designated No Parking zones or sitting on the curb where standing is not allowed.

Should you have any questions, please email us at roundfile@localhost, or contact school administration.

Thank you and we appreciate everyone's cooperation with this safety initiative,

ACME Management
"""

print_chat(f"Summarize: {document}")

In [ ]:
print_chat("How do you say welcome to dinner in Spanish?")